In [2]:
import requests
from datetime import datetime, timedelta
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import johnsonsu
from scipy.optimize import minimize
import csv
import pandas as pd

In [48]:
key='&apikey=ZKMMTO1ATDBLXH2K' # API Key
ticker='&symbol=SPY' # Ticker
endpoint='function=TIME_SERIES_DAILY_ADJUSTED' # Called 'function', the dataset we want
size='&outputsize=full'
web='https://www.alphavantage.co/query?'
url =web+endpoint+ticker+size+key

r = requests.get(url)
print(r.status_code) # 200 good, 400 bad
data = r.json()

print(data.keys()) #printing the keys
meta = data['Meta Data']
time_series_data = data['Time Series (Daily)']

ts_df = pd.DataFrame.from_dict(time_series_data, orient='index').reset_index().rename(columns={'index': 'Date'})
clean_cols_dict = {'1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', # Dictionary to convert the names of the columns
            '5. adjusted close': 'Adj Close', '6. volume': 'Volume', '7. dividend amount': 'Dividend', '8. split coefficient': 'Split Coef'}
ts_df = ts_df.rename(columns=clean_cols_dict)

clean=[] # This is how you do it with standard Python
for date in time_series_data.keys():
    # print (r1[date])
    clean.append([date, meta['2. Symbol'], time_series_data[date]['4. close']]) #stacking prices


ts_np = ts_df.to_numpy() # How to convert a df to a 2d numpy matrix
ts_df = ts_df.sort_values(by='Date', ascending=True)
ts_df['Adj Close'] = ts_df['Adj Close'].astype(float)
ts_df['Date'] = pd.to_datetime(ts_df['Date'])
ts_df

200
dict_keys(['Meta Data', 'Time Series (Daily)'])


,Date,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Coef
6595,1999-11-01,136.5,137.0,135.5625,135.5625,85.122145,4006500,0.0000,1.0
6594,1999-11-02,135.968704,137.25,134.593704,134.593704,84.513820,6516900,0.0000,1.0
6593,1999-11-03,136.0,136.375,135.125,135.5,85.082900,7222300,0.0000,1.0
6592,1999-11-04,136.75,137.359299,135.765594,136.531204,85.730412,7907500,0.0000,1.0
6591,1999-11-05,138.625,139.109299,136.781204,137.875,86.574205,7431500,0.0000,1.0
...,...,...,...,...,...,...,...,...,...
4,2026-01-15,694.57,695.45,691.25,692.24,692.240000,77861974,0.0000,1.0
3,2026-01-16,693.66,694.25,690.1,691.66,691.660000,79289200,0.0000,1.0
2,2026-01-20,681.49,684.77,676.57,677.58,677.580000,107178877,0.0000,1.0
1,2026-01-21,679.65,688.735,678.13,685.4,685.400000,127844467,0.0000,1.0


In [49]:
# 2. Set the date column as the index (required for resampling)
monthly_df = ts_df.set_index('Date')

# 3. Create a Series of just the Month-End prices
# 'ME' stands for Month End. (Use 'M' if you are on an older version of Pandas)
# Instead of resample, use groupby with a Grouper
# tail(1) grabs the last actual row of the month, preserving its original Date index
monthly_prices = monthly_df.groupby(pd.Grouper(freq='ME')).tail(1)['Adj Close']

# 4. Calculate the Log Return
# Formula: ln(Current Month Price) - ln(Previous Month Price)
monthly_log_returns = np.log(monthly_prices) - np.log(monthly_prices.shift(1))

# Optional: Put it into a readable DataFrame
monthly_summary = pd.DataFrame({
    'Month End Price': monthly_prices,
    'Monthly Log Return': monthly_log_returns
})

monthly_summary = monthly_summary[['Monthly Log Return']].reset_index()
monthly_summary

,Date,Monthly Log Return
0,1999-11-30,NaN
1,1999-12-31,0.055520
2,2000-01-31,-0.051069
3,2000-02-29,-0.015343
4,2000-03-31,0.092483
...,...,...
310,2025-09-30,0.034987
311,2025-10-31,0.023558
312,2025-11-28,0.001948
313,2025-12-31,0.000771


In [50]:
ts_df = pd.merge(ts_df, monthly_summary, on='Date', how='left')


In [51]:
threshold = ts_df['Monthly Log Return'].quantile(0.025)
buy = []

for index, row in ts_df.iterrows():
    if pd.isna(row['Monthly Log Return']):
        buy.append(0)
        continue
    elif row['Monthly Log Return'] < threshold:
        buy.append(1)
    else:
        buy.append(0)

ts_df['Buy'] = buy
ts_df['Sell'] = ts_df['Buy'].shift(1)

buy_price = 0

returns = []
for index, row in ts_df.iterrows():
    if row['Buy'] == 1:
        buy_price = row['Adj Close']
        returns.append(0)
    elif row['Sell'] == 1:
        sell_price = row['Adj Close']
        day_return = np.log(sell_price) - np.log(buy_price)
        returns.append(day_return)
    else:
        returns.append(0)

ts_df['Strategy Return'] = returns


In [ ]:
strategy_df = ts_df[(ts_df['Buy'] == 1) | (ts_df['Sell'] == 1)]
returns_df = ts_df[ts_df['Sell'] == 1]



,Date,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Coef,Monthly Log Return,Buy,Sell,Strategy Return
335,2001-03-01,124.050003,124.599998,121.75,124.599998,79.264735,14672000,0.0000,1.0,NaN,0,1.0,0.005230
731,2002-10-01,82.43,85.769997,81.470001,85.720001,55.835258,67198100,0.0000,1.0,NaN,0,1.0,0.046931
2242,2008-10-01,115.269997,116.690002,113.949997,116.059998,84.341352,332783000,0.0000,1.0,NaN,0,1.0,0.000603
2265,2008-11-03,96.779999,97.690002,95.949997,97.110001,70.570299,205419400,0.0000,1.0,NaN,0,1.0,0.002887
2345,2009-03-02,72.519997,73.919998,70.370003,70.599998,51.723837,426452600,0.0000,1.0,NaN,0,1.0,-0.046089
4822,2019-01-02,245.98,251.21,245.95,250.18,224.868305,126925199,0.0000,1.0,NaN,0,1.0,0.001040
5136,2020-04-01,247.98,257.66,243.9,246.15,226.816709,188373330,0.0000,1.0,NaN,0,1.0,-0.046049
5767,2022-10-03,361.08,368.55,359.21,366.61,350.756481,89756479,0.0000,1.0,NaN,0,1.0,0.026059


In [59]:
x = returns_df['Strategy Return'].tolist()
t_stat, p_val = stats.ttest_1samp(x, 0) 
t_stat, p_val

(np.float64(-0.10382260127307678), np.float64(0.9202221275277256))

In [ ]:
# log(a) - log(b) = log(a/b)
ts_df['Monthly Log Return'] = np.log(ts_df['Adj Close'] / ts_df['Adj Close'].shift(1))
p2_5 = ts_df['Monthly Log Return'].quantile(0.025)


-0.09189884660357991
